In [1]:
library("DESeq2")
library("geneplotter")
library("ggplot2")
library("vsn")
library("genefilter")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min


Attaching packag

In [2]:
setwd("/home/gstupp/projects/Wolan/ana_extractions/analysis/analysis_deseq")

In [3]:
file_path = "X.csv"
countData = read.csv(file_path, row.names = 1)

In [4]:
meta_path = "metadata.csv"
sampleTable = read.csv(meta_path, row.names = 1)
sampleTable = sampleTable[colnames(countData),]
sampleTable$cell = factor(sampleTable$cell)
sampleTable$method = factor(sampleTable$method)
sampleTable$method <- relevel(sampleTable$method, ref="triflic")
sampleTable = sampleTable[!(rownames(sampleTable) %in% c("PA_pellet_1", "PA_pellet_2", "PA_pellet_3")),]
sampleTable

,cell,file,method,name.1,pretty_name
BS_sonic_1,BS,data/BS/Sonic/BS sonic 1.txt,sonic,BS_sonic_1,BS sonic 1
BS_sonic_2,BS,data/BS/Sonic/BS sonic 2.txt,sonic,BS_sonic_2,BS sonic 2
BS_sonic_3,BS,data/BS/Sonic/BS sonic 3.txt,sonic,BS_sonic_3,BS sonic 3
BS_triflic_1,BS,data/BS/Triflic/BS triflic 1.txt,triflic,BS_triflic_1,BS triflic 1
BS_triflic_2,BS,data/BS/Triflic/BS triflic 2.txt,triflic,BS_triflic_2,BS triflic 2
BS_triflic_3,BS,data/BS/Triflic/BS triflic 3.txt,triflic,BS_triflic_3,BS triflic 3
Jurkat_sonic_1,Jurkat,data/Jurkat/Sonic/Jurkat sonic 1.txt,sonic,Jurkat_sonic_1,Jurkat sonic 1
Jurkat_sonic_2,Jurkat,data/Jurkat/Sonic/Jurkat sonic 2.txt,sonic,Jurkat_sonic_2,Jurkat sonic 2
Jurkat_sonic_3,Jurkat,data/Jurkat/Sonic/Jurkat sonic 3.txt,sonic,Jurkat_sonic_3,Jurkat sonic 3
Jurkat_triflic_1,Jurkat,data/Jurkat/Triflic/Jurkat triflic 1.txt,triflic,Jurkat_triflic_1,Jurkat triflic 1


## BS

In [5]:
sampleTableX = sampleTable[sampleTable$cell=="BS",]
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 1, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,BS_sonic_1,BS_sonic_2,BS_sonic_3,BS_triflic_1,BS_triflic_2,BS_triflic_3
C0H3V2,4,7,5,0,3,4
C0H3V3,6,6,9,0,0,0


In [6]:
dds = DESeq(dds)
res = results(dds, alpha=0.05)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 1490 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)     : 184, 12% 
LFC < 0 (down)   : 213, 14% 
outliers [1]     : 0, 0% 
low counts [2]   : 462, 31% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [7]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="BS_deseq_results.csv")
resOrdered[0:20,]

log2 fold change (MLE): method sonic vs triflic 
Wald test p-value: method sonic vs triflic 
DataFrame with 20 rows and 6 columns
        baseMean log2FoldChange     lfcSE      stat       pvalue         padj
       <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
P80239  747.5681       4.366532 0.2987603  14.61550 2.236998e-48 2.427142e-45
P54375  329.4090      -5.712130 0.4305659 -13.26656 3.618028e-40 1.962780e-37
P0CI73  174.4533      -5.838118 0.4605166 -12.67732 7.898288e-37 2.856547e-34
P26901  280.4514       3.304183 0.2784145  11.86786 1.738671e-32 4.716144e-30
P94391  180.9387       4.242991 0.3594748  11.80331 3.752764e-32 8.143498e-30
...          ...            ...       ...       ...          ...          ...
P94550  47.60762       4.011827 0.5316912  7.545408 4.508736e-14 3.057487e-12
P54542  59.62276       3.447174 0.4605396  7.485075 7.150626e-14 4.563782e-12
P45745  61.68154      -9.307488 1.2457836 -7.471192 7.947178e-14 4.655357e-12
P54716  48.0

## PA

In [8]:
sampleTableX = sampleTable[sampleTable$cell=="PA",]
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 1, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,PA_sonic_1,PA_sonic_2,PA_sonic_3,PA_triflic_1,PA_triflic_2,PA_triflic_3
E1JGJ8,7,2,0,0,0,0
G3XCT6,37,12,6,13,35,18


In [9]:
dds = DESeq(dds)
res = results(dds, alpha=0.05)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 2054 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)     : 19, 0.93% 
LFC < 0 (down)   : 65, 3.2% 
outliers [1]     : 0, 0% 
low counts [2]   : 1035, 50% 
(mean count < 3)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [10]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="PA_deseq_results.csv")
resOrdered[0:20,]

log2 fold change (MLE): method sonic vs triflic 
Wald test p-value: method sonic vs triflic 
DataFrame with 20 rows and 6 columns
         baseMean log2FoldChange     lfcSE       stat       pvalue         padj
        <numeric>      <numeric> <numeric>  <numeric>    <numeric>    <numeric>
Q9HWW1  556.09650      -5.233437 0.5091475 -10.278822 8.779375e-25 9.990929e-22
P13794  546.29212      -7.082614 0.7110108  -9.961331 2.250297e-23 1.280419e-20
Q9HVI2  301.44015     -11.803714 1.3073315  -9.028861 1.734677e-19 6.580210e-17
Q9HUM0  147.19566      -7.388025 0.8303335  -8.897659 5.703681e-19 1.622697e-16
Q9I4S1   49.57822      -6.486841 0.9827601  -6.600635 4.094003e-11 9.317950e-09
...           ...            ...       ...        ...          ...          ...
P25254   9.496732      -6.818542 1.5720956  -4.337231 1.442890e-05  0.001026255
O82851 228.254988      -2.531163 0.5983835  -4.230001 2.336906e-05  0.001564352
Q9HU56 118.489780      -2.949748 0.7092333  -4.159066 3.195515e-05  0.

## Jurkat

In [11]:
sampleTableX = sampleTable[sampleTable$cell=="Jurkat",]
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 1, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,Jurkat_sonic_1,Jurkat_sonic_2,Jurkat_sonic_3,Jurkat_triflic_1,Jurkat_triflic_2,Jurkat_triflic_3
A0A024QZP7,0,0,6,0,0,0
A0A024R571,4,6,6,6,5,2


In [12]:
dds = DESeq(dds)
res = results(dds, alpha=0.05)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 3762 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)     : 3, 0.08% 
LFC < 0 (down)   : 1, 0.027% 
outliers [1]     : 21, 0.56% 
low counts [2]   : 0, 0% 
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [13]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="Jurkat_deseq_results.csv")
resOrdered[0:20,]

log2 fold change (MLE): method sonic vs triflic 
Wald test p-value: method sonic vs triflic 
DataFrame with 20 rows and 6 columns
         baseMean log2FoldChange     lfcSE      stat       pvalue         padj
        <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
Q7L7L0   14.96284     -21.884964  2.639984 -8.289810 1.134298e-16 6.862504e-14
Q9BVA1  104.95902       9.760908  1.898267  5.142009 2.718158e-07 8.222428e-05
E9PQB7   66.68883       9.106474  1.966885  4.629896 3.658494e-06 7.377963e-04
P04075   44.71724       8.529568  1.920425  4.441500 8.933395e-06 1.351176e-03
F5GWF6   12.16857      -7.478031  1.995232 -3.747950 1.782855e-04 1.797712e-02
...           ...            ...       ...       ...          ...          ...
M0R2I7  25.463798      -4.306831  1.497472 -2.876068  0.004026629    0.1595431
Q02750   7.892063       6.023803  2.114806  2.848395  0.004394037    0.1595431
Q13011   7.205527       5.894681  2.093610  2.815559  0.004869253    0.1636610
E